In [ ]:
import pandas as pd
import panel as pn

pn.extension()

The ``DataFrame`` widget allows displaying and editing a pandas DataFrame.

For more information about listening to widget events and laying out widgets refer to the [widgets user guide](../../user_guide/Widgets.ipynb). Alternatively you can learn how to build GUIs by declaring parameters independently of any specific widgets in the [param user guide](../../user_guide/Param.ipynb). To express interactivity entirely using Javascript without the need for a Python server take a look at the [links user guide](../../user_guide/Param.ipynb).

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **``editors``** (``dict``):  A dictionary mapping from column name to a bokeh CellEditor instance, which overrides the default.
* **``fit_columns``** (``boolean``, default=True): Whether columns should expand to the available width. 
* **``formatters``** (``dict``): A dictionary mapping from column name to a bokeh CellFormatter instance, which overrides the default.
* **``row_height``** (``int``): The height of each table row.
* **``selection``** (``list``) The currently selected rows 
* **``value``** (``pd.DataFrame``): The pandas DataFrame to display and edit
* **``widths``** (``dict``): A dictionary mapping from column name to column width in the rendered table.

##### Display

* **``disabled``** (boolean): Whether the widget is editable
* **``name``** (str): The title of the widget

___

The ``DataFrame`` widget renders an table which allows directly editing the contents of the dataframe with any changes being synced with Python. Note that it modifies the ``pd.DataFrame`` in place.

In [ ]:
df = pd.DataFrame({'int': [1, 2, 3], 'float': [3.14, 6.28, 9.42], 'str': ['A', 'B', 'C']}, index=[1, 2, 3])

df_widget = pn.widgets.DataFrame(df, name='DataFrame')

df_widget

By default the widget will pick bokeh ``CellFormatter`` and ``CellEditor`` types appropriate to the dtype of the column. These may be overriden by explicit dictionaries mapping from the column name to the editor or formatter instance. For example below we create a ``SelectEditor`` instance to pick from four options in the ``str`` column and a ``NumberFormatter`` to customize the formatting of the float values:

In [ ]:
from bokeh.models.widgets.tables import SelectEditor, NumberFormatter

editor = SelectEditor(options=['A', 'B', 'C', 'D'])
formatter = NumberFormatter(format='0.00000') 

table = pn.widgets.DataFrame(df, editors={'str': editor}, formatters={'float': formatter})
table

Once initialized the ``selection`` property will return the integer indexes of the selected rows and the ``selected_dataframe`` property will return a new DataFrame containing just the selected rows:

In [ ]:
table.selection = [0, 2]

table.selected_dataframe

### Controls

The `DataFrame` widget exposes a number of options which can be changed from both Python and Javascript. Try out the effect of these parameters interactively:

In [ ]:
pn.Row(df_widget.controls(jslink=True), df_widget)